In [39]:
import pandas as pd
import numpy as np
import re
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
problem = pd.read_csv("data/для анализа/dataset._problems.csv", sep=";")
normal = pd.read_csv("data/для анализа/dataset._normal.csv", sep=";")

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/1852484758.py:1: DtypeWarning: Columns (3,11,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  problem = pd.read_csv("data/для анализа/dataset._problems.csv", sep=";")
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/1852484758.py:2: DtypeWarning: Columns (3,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  normal = pd.read_csv("data/для анализа/dataset._normal.csv", sep=";")


In [4]:
def nan_handler(df: pd.DataFrame):
    datetime = ['Дата и время']
    zero_data = [
        'Нагрузка на двигатель',
        'Стояночный тормоз',
        'Засоренность фильтра рулевого управления',
        'Засоренность фильтра навесного оборудования',
        'Отопитель',
        'Выход блока управления двигателем',
        'Включение тормозков',
        'Засоренность фильтра слива',
        'Аварийное давление масла КПП',
        'Аварийная температура масла ДВС',
        'Неисправность тормозной системы',
        'Термостарт',
        'Разрешение запуска двигателя',
        'Низкий уровень ОЖ',
        'Аварийная температура масла ГТР',
        'Необходимость сервисного обслуживания',
        'Подогрев топливного фильтра',
        'Вода в топливе',
        'Холодный старт',
        'Крутящий момент' ,
        'Положение рейки ТНВД' ,
        'Расход топлива' ,
        'Давление наддувочного воздуха двигателя',
        'Температура масла гидравлики' ,
        'Педаль слива',
        'iButton2']
    
    regex = '|'.join(map(re.escape, zero_data))  # Escape special characters
    print("Regex pattern:", regex)

    # Check if columns exist before dropping
    columns_to_drop = df.filter(regex=regex).columns
    if not columns_to_drop.empty:
        df = df.drop(columns_to_drop, axis=1)

    for col in df.columns:
        if col in datetime:
            df[col] = pd.to_datetime(df[col], format='%d/%m/%Y %H:%M:%S')
        if col not in datetime and df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace(',', '.').str.replace(':', '.')
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

In [5]:
problem = nan_handler(problem)
normal = nan_handler(normal)

Regex pattern: Нагрузка\ на\ двигатель|Стояночный\ тормоз|Засоренность\ фильтра\ рулевого\ управления|Засоренность\ фильтра\ навесного\ оборудования|Отопитель|Выход\ блока\ управления\ двигателем|Включение\ тормозков|Засоренность\ фильтра\ слива|Аварийное\ давление\ масла\ КПП|Аварийная\ температура\ масла\ ДВС|Неисправность\ тормозной\ системы|Термостарт|Разрешение\ запуска\ двигателя|Низкий\ уровень\ ОЖ|Аварийная\ температура\ масла\ ГТР|Необходимость\ сервисного\ обслуживания|Подогрев\ топливного\ фильтра|Вода\ в\ топливе|Холодный\ старт|Крутящий\ момент|Положение\ рейки\ ТНВД|Расход\ топлива|Давление\ наддувочного\ воздуха\ двигателя|Температура\ масла\ гидравлики|Педаль\ слива|iButton2
Regex pattern: Нагрузка\ на\ двигатель|Стояночный\ тормоз|Засоренность\ фильтра\ рулевого\ управления|Засоренность\ фильтра\ навесного\ оборудования|Отопитель|Выход\ блока\ управления\ двигателем|Включение\ тормозков|Засоренность\ фильтра\ слива|Аварийное\ давление\ масла\ КПП|Аварийная\ температура

In [6]:
merge = pd.concat([problem, normal], axis = 0)

In [7]:
important_params = {
    'ДВС': 'ДВС. Температура охлаждающей жидкости',
    'КПП': 'КПП. Температура масла',
    'Гидравлика': 'Аварийная температура масла в гидросистеме (spn3849)',
    'Тормозная': 'Давление в пневмостистеме (spn46), кПа',
    'Электросистема': 'Электросистема. Напряжение'
}

tractor_params = {
    'ДВС': ['Полож.пед.акселер.,%', 'Давл.масла двиг.,кПа', 
        'Обор.двиг.,об/мин', 'Значение счетчика моточасов, час:мин', 'ДВС. Частота вращения коленчатого вала', 'ДВС. Давление смазки',
        'ДВС. Температура охлаждающей жидкости',  'Аварийная температура охлаждающей жидкости (spn3841)', 'Засоренность воздушного фильтра (spn3840)',
        'Аварийное давление масла ДВС (spn3846)', 'Засоренность фильтра ДВС (spn3845)', 'ДВС. Температура наддувочного воздуха, °С', 
        ],
    'КПП': ['КПП. Температура масла', 'КПП. Давление масла в системе смазки', 'Сост.пед.сцепл.', 'Скорость', 
            'Нейтраль КПП (spn3843)', 'Засоренность фильтра КПП (spn3847)', 'Текущая передача (spn523)'],
    'Гидравлика': ['Недопустимый уровень масла в гидробаке (spn3850)', 'Аварийная температура масла в гидросистеме (spn3849)'],
    'Тормозная': ['Давление в пневмостистеме (spn46), кПа', 'Аварийное давление в I контуре тормозной системы (spn3848)',
                   'Аварийное давление в II контуре тормозной системы (spn3855)'],
    'Электросистема': ['Электросистема. Напряжение', 'Зарядка АКБ (spn3854)'],
}

important_components = {
    'ICE': 'Engine Coolant Temperature',
    'Transmission': 'Transmission Oil Temperature',
    'Hydraulics': 'Emergency Oil Temperature in Hydraulic System',
    'Brake System': 'Pneumatic System Pressure',
    'Electrical System': 'Electrical System Voltage'
}



In [125]:
time_series = merge["Дата и время"]

engine = merge[tractor_params["ДВС"]]
transmission =  merge[tractor_params["КПП"]]
hydraulics = merge[tractor_params["Гидравлика"]]
brake = merge[tractor_params["Тормозная"]]
electric = merge[tractor_params["Электросистема"]]

In [132]:
engine["ds"] = time_series
transmission["ds"] = time_series
hydraulics["ds"] = time_series
brake["ds"] = time_series
electric["ds"] = time_series

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/989031027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engine["ds"] = time_series
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/989031027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transmission["ds"] = time_series
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/989031027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [10]:
drop=engine.isna().sum().index[7:11]

In [11]:
engine.isna().sum() / len(engine)

Полож.пед.акселер.,%                                    0.169019
Давл.масла двиг.,кПа                                    0.169133
Обор.двиг.,об/мин                                       0.170136
Значение счетчика моточасов, час:мин                    0.775761
ДВС. Частота вращения коленчатого вала                  0.168799
ДВС. Давление смазки                                    0.169133
ДВС. Температура охлаждающей жидкости                   0.170973
Аварийная температура охлаждающей жидкости (spn3841)    0.170527
Засоренность воздушного фильтра (spn3840)               0.170527
Аварийное давление масла ДВС (spn3846)                  0.170527
Засоренность фильтра ДВС (spn3845)                      0.170527
ДВС. Температура наддувочного воздуха, °С               1.000000
ds                                                      0.000000
dtype: float64

In [12]:
drop

Index(['Аварийная температура охлаждающей жидкости (spn3841)', 'Засоренность воздушного фильтра (spn3840)', 'Аварийное давление масла ДВС (spn3846)', 'Засоренность фильтра ДВС (spn3845)'], dtype='object')

In [13]:
engine.drop(columns=drop, inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/511740602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engine.drop(columns=drop, inplace=True)


In [14]:
def print_analyze(df):

    for column in df.columns:
        total_values = len(df[column])  # Общее количество значений в колонке
        nan_count = df[column].isna().sum()  # Количество NaN значений
        nan_share = nan_count / total_values  # Доля NaN значений

        # Исключаем NaN значения для поиска самого популярного значения
        most_frequent_value = df[column].dropna().mode()[0] if not df[column].dropna().empty else 'Все значения - NaN'
        
        print(f'Колонка {column}:')
        print(f'    Самое популярное значение (не считая NaN): {most_frequent_value}')
        print(f'    Доля NaN значений: {nan_share:.5f}')



In [15]:
print_analyze(engine)

Колонка Полож.пед.акселер.,%:
    Самое популярное значение (не считая NaN): 0.0
    Доля NaN значений: 0.16902
Колонка Давл.масла двиг.,кПа:
    Самое популярное значение (не считая NaN): 520.0
    Доля NaN значений: 0.16913
Колонка Обор.двиг.,об/мин:
    Самое популярное значение (не считая NaN): 1900.75
    Доля NaN значений: 0.17014
Колонка Значение счетчика моточасов, час:мин:
    Самое популярное значение (не считая NaN): 507.0
    Доля NaN значений: 0.77576
Колонка ДВС. Частота вращения коленчатого вала:
    Самое популярное значение (не считая NaN): 1900.75
    Доля NaN значений: 0.16880
Колонка ДВС. Давление смазки:
    Самое популярное значение (не считая NaN): 520.0
    Доля NaN значений: 0.16913
Колонка ДВС. Температура охлаждающей жидкости:
    Самое популярное значение (не считая NaN): 76.0
    Доля NaN значений: 0.17097
Колонка ДВС. Температура наддувочного воздуха, °С:
    Самое популярное значение (не считая NaN): Все значения - NaN
    Доля NaN значений: 1.00000
Колон

In [123]:
def fillna_mean(df):
    for column in df.columns:
        mean_value = df[column].mean()
        df[column].fillna(mean_value, inplace=True)
        

In [16]:
engine.drop(columns=["ДВС. Температура наддувочного воздуха, °С", "Значение счетчика моточасов, час:мин"], inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/2822520260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engine.drop(columns=["ДВС. Температура наддувочного воздуха, °С", "Значение счетчика моточасов, час:мин"], inplace=True)


In [18]:
fillna_mean(engine)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/1356920205.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/1356920205.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(mean_value, inplace=True)


In [19]:
engine

,"Полож.пед.акселер.,%","Давл.масла двиг.,кПа","Обор.двиг.,об/мин",ДВС. Частота вращения коленчатого вала,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,ds
0,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-08-01 11:45:47
1,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-08-01 11:46:01
2,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-08-01 12:23:43
3,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-08-01 12:23:57
4,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-08-01 12:36:35
...,...,...,...,...,...,...,...
892878,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-05-31 04:03:37
892879,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-05-31 07:06:50
892880,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-05-31 06:05:47
892881,34.142771,420.989907,1599.820497,1599.834778,420.997693,75.039454,2023-05-31 08:07:59


In [20]:
engine.rename(columns={
    "ДВС. Температура охлаждающей жидкости":"y"
}, inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/3829084221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engine.rename(columns={


In [21]:
engine.sort_values(by="ds", inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/3011831549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engine.sort_values(by="ds", inplace=True)


In [115]:
engine_train = engine.iloc[:100000, :]
engine_train["ds"].diff()

524296               NaT
820842   0 days 00:00:10
524297   0 days 00:00:20
820843   0 days 00:00:10
524298   0 days 00:00:20
               ...      
580349   0 days 00:00:10
580350   0 days 00:00:20
580351   0 days 00:00:21
580352   0 days 00:00:06
580353   0 days 00:00:03
Name: ds, Length: 100000, dtype: timedelta64[ns]

In [116]:
split_point = int(len(engine_train) * 0.8)

# Делим на тренировочный и тестовый наборы
train_df = engine_train.iloc[:split_point]
test_df = engine_train.iloc[split_point:]

# Выводим размеры тренировочного и тестового наборов
print(f'Train set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Train set size: 80000
Test set size: 20000


In [117]:
model = Prophet(changepoint_prior_scale=0.1)

model.fit(train_df)  

02:48:43 - cmdstanpy - INFO - Chain [1] start processing
02:48:58 - cmdstanpy - INFO - Chain [1] done processing


In [121]:
 # Пример с предсказанием на 1 год вперед
forecast = model.predict(test_df)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-05-12 08:05:51,74.563016,66.702687,80.098770,74.563016,74.563016,-1.137999,-1.137999,-1.137999,-1.137999,-1.137999,-1.137999,0.0,0.0,0.0,73.425016
1,2023-05-12 08:06:09,74.562915,65.958443,79.928843,74.562915,74.562915,-1.130961,-1.130961,-1.130961,-1.130961,-1.130961,-1.130961,0.0,0.0,0.0,73.431954
2,2023-05-12 08:06:15,74.562882,66.378668,80.501899,74.562882,74.562882,-1.128616,-1.128616,-1.128616,-1.128616,-1.128616,-1.128616,0.0,0.0,0.0,73.434266
3,2023-05-12 08:06:21,74.562849,66.877725,79.829712,74.562849,74.562849,-1.126271,-1.126271,-1.126271,-1.126271,-1.126271,-1.126271,0.0,0.0,0.0,73.436578
4,2023-05-12 08:06:45,74.562715,66.655199,80.122919,74.562715,74.562715,-1.116895,-1.116895,-1.116895,-1.116895,-1.116895,-1.116895,0.0,0.0,0.0,73.445820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2023-05-14 19:20:56,73.375741,52.916141,97.438803,53.119427,95.734330,0.528761,0.528761,0.528761,0.528761,0.528761,0.528761,0.0,0.0,0.0,73.904502
19996,2023-05-14 19:21:16,73.375630,52.982805,97.885893,53.118009,95.735668,0.532069,0.532069,0.532069,0.532069,0.532069,0.532069,0.0,0.0,0.0,73.907698
19997,2023-05-14 19:21:37,73.375513,53.145563,96.204559,53.116585,95.737000,0.535542,0.535542,0.535542,0.535542,0.535542,0.535542,0.0,0.0,0.0,73.911054
19998,2023-05-14 19:21:43,73.375479,53.047279,96.947987,53.115244,95.738416,0.536534,0.536534,0.536534,0.536534,0.536534,0.536534,0.0,0.0,0.0,73.912013


In [120]:
def calculate_forecast_accuracy(y_true, y_pred):
    """
    Рассчитывает метрики точности прогноза: MAE, MSE и MAPE.

    Параметры:
    - y_true (array-like): Фактические значения.
    - y_pred (array-like): Прогнозируемые значения.

    Возвращает:
    - dict: Словарь с метриками точности: MAE, MSE, MAPE.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return {
        'MAE': mae,
        'MSE': mse,
        'MAPE': mape
    }



In [122]:
print(calculate_forecast_accuracy(test_df["y"], forecast["yhat"]))

{'MAE': 4.029057979212733, 'MSE': 38.19729163512326, 'MAPE': nan}


In [126]:
transmission_drop = ["Сост.пед.сцепл.", "Текущая передача (spn523)"]
transmission.drop(columns=transmission_drop, inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/3243942236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transmission.drop(columns=transmission_drop, inplace=True)


In [127]:
transmission

,КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,Нейтраль КПП (spn3843),Засоренность фильтра КПП (spn3847)
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
892878,NaN,NaN,NaN,NaN,NaN
892879,NaN,NaN,NaN,NaN,NaN
892880,NaN,NaN,NaN,NaN,NaN
892881,NaN,NaN,NaN,NaN,NaN


In [128]:
fillna_mean(transmission)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(mean_value, inplace=True)


In [130]:
transmission.rename(columns={
    "КПП. Давление масла в системе смазки":"y"
}, inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/1694111974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transmission.rename(columns={


In [133]:
transmission

,КПП. Температура масла,y,Скорость,Нейтраль КПП (spn3843),Засоренность фильтра КПП (spn3847),ds
0,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-08-01 11:45:47
1,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-08-01 11:46:01
2,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-08-01 12:23:43
3,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-08-01 12:23:57
4,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-08-01 12:36:35
...,...,...,...,...,...,...
892878,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-05-31 04:03:37
892879,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-05-31 07:06:50
892880,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-05-31 06:05:47
892881,-272.999971,1193.203212,5.764284,0.072742,0.000002,2023-05-31 08:07:59


In [135]:
transmission_train = transmission.iloc[:100000, :]
split_point = int(len(transmission_train) * 0.8)

# Делим на тренировочный и тестовый наборы
train_df = transmission_train.iloc[:split_point]
test_df = transmission_train.iloc[split_point:]

# Выводим размеры тренировочного и тестового наборов
print(f'Train set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Train set size: 80000
Test set size: 20000


In [140]:
model = Prophet()
model.fit(train_df)
forecast = model.predict(test_df) 
print(calculate_forecast_accuracy(test_df["y"], forecast["yhat"]))

03:03:08 - cmdstanpy - INFO - Chain [1] start processing
03:03:26 - cmdstanpy - INFO - Chain [1] done processing


{'MAE': 88.51478477485239, 'MSE': 26929.59115738276, 'MAPE': nan}


In [142]:
print_analyze(hydraulics)

Колонка Недопустимый уровень масла в гидробаке (spn3850):
    Самое популярное значение (не считая NaN): 0.0
    Доля NaN значений: 0.17053
Колонка Аварийная температура масла в гидросистеме (spn3849):
    Самое популярное значение (не считая NaN): 0.0
    Доля NaN значений: 0.17053
Колонка ds:
    Самое популярное значение (не считая NaN): 2023-06-06 11:57:39
    Доля NaN значений: 0.00000


In [143]:
fillna_mean(hydraulics)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(mean_value, inplace=True)


In [146]:
hydraulics.drop(columns="Недопустимый уровень масла в гидробаке (spn3850)", inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/2142650970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydraulics.drop(columns="Недопустимый уровень масла в гидробаке (spn3850)", inplace=True)


In [148]:
hydraulics.rename(columns={
    "Аварийная температура масла в гидросистеме (spn3849)":"y"
}, inplace=True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/442822924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydraulics.rename(columns={


In [150]:
hydraulics_train = hydraulics.iloc[:100000, :]
split_point = int(len(transmission_train) * 0.8)

# Делим на тренировочный и тестовый наборы
train_df = hydraulics_train.iloc[:split_point]
test_df = hydraulics_train.iloc[split_point:]

# Выводим размеры тренировочного и тестового наборов
print(f'Train set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Train set size: 80000
Test set size: 20000


In [170]:
hydraulics_train["y"].value_counts()

y
0.000000    56913
0.000001    43084
1.000000        3
Name: count, dtype: int64

In [151]:
model = Prophet()
model.fit(train_df)
forecast = model.predict(test_df) 
print(calculate_forecast_accuracy(test_df["y"], forecast["yhat"]))

03:12:49 - cmdstanpy - INFO - Chain [1] start processing
03:12:50 - cmdstanpy - INFO - Chain [1] done processing


{'MAE': 0.00018605608310575446, 'MSE': 6.729626930308886e-08, 'MAPE': nan}


In [153]:
fillna_mean(brake)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(mean_value, inplace=True)


In [154]:
brake

,"Давление в пневмостистеме (spn46), кПа",Аварийное давление в I контуре тормозной системы (spn3848),Аварийное давление в II контуре тормозной системы (spn3855),ds
0,762.02464,0.001787,0.001379,2023-08-01 11:45:47
1,762.02464,0.001787,0.001379,2023-08-01 11:46:01
2,762.02464,0.001787,0.001379,2023-08-01 12:23:43
3,762.02464,0.001787,0.001379,2023-08-01 12:23:57
4,762.02464,0.001787,0.001379,2023-08-01 12:36:35
...,...,...,...,...
892878,762.02464,0.001787,0.001379,2023-05-31 04:03:37
892879,762.02464,0.001787,0.001379,2023-05-31 07:06:50
892880,762.02464,0.001787,0.001379,2023-05-31 06:05:47
892881,762.02464,0.001787,0.001379,2023-05-31 08:07:59


In [156]:
brake["Давление в пневмостистеме (spn46), кПа"].value_counts()

Давление в пневмостистеме (spn46), кПа
762.02464    672444
760.00000    648363
784.00000    236655
768.00000    202683
816.00000    155507
728.00000    151170
808.00000    149894
776.00000    124016
720.00000    111135
744.00000    104461
696.00000     95264
800.00000     89955
840.00000     79610
824.00000     58260
848.00000     50825
704.00000     35419
736.00000     34271
688.00000     31612
680.00000     30796
664.00000     16377
656.00000     14486
648.00000     11960
640.00000      7167
624.00000      6825
616.00000      5372
608.00000      3842
600.00000      3610
568.00000      2298
584.00000      2231
528.00000      1818
544.00000      1599
464.00000      1565
576.00000      1536
536.00000      1524
832.00000      1498
496.00000      1390
560.00000      1345
520.00000      1320
480.00000      1265
504.00000       822
488.00000       670
384.00000       269
448.00000       223
456.00000       222
440.00000       218
416.00000       189
424.00000       145
408.00000       144
4

In [157]:
brake.rename(columns={
    "Давление в пневмостистеме (spn46), кПа":"y"
}, inplace = True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/217077515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brake.rename(columns={


In [158]:
brake_train = brake.iloc[:100000, :]
split_point = int(len(transmission_train) * 0.8)

# Делим на тренировочный и тестовый наборы
train_df = brake_train.iloc[:split_point]
test_df = brake_train.iloc[split_point:]

# Выводим размеры тренировочного и тестового наборов
print(f'Train set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Train set size: 80000
Test set size: 20000


In [159]:
model = Prophet()
model.fit(train_df)
forecast = model.predict(test_df) 
print(calculate_forecast_accuracy(test_df["y"], forecast["yhat"]))

03:20:41 - cmdstanpy - INFO - Chain [1] start processing
03:20:52 - cmdstanpy - INFO - Chain [1] done processing


{'MAE': 40.16773030745285, 'MSE': 1727.1634698980401, 'MAPE': nan}


In [161]:
fillna_mean(electric)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/325840709.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(mean_value, inplace=True)


In [162]:
electric

,Электросистема. Напряжение,Зарядка АКБ (spn3854),ds
0,27.341695,0.003046,2023-08-01 11:45:47
1,27.341695,0.003046,2023-08-01 11:46:01
2,27.341695,0.003046,2023-08-01 12:23:43
3,27.341695,0.003046,2023-08-01 12:23:57
4,27.341695,0.003046,2023-08-01 12:36:35
...,...,...,...
892878,27.341695,0.003046,2023-05-31 04:03:37
892879,27.341695,0.003046,2023-05-31 07:06:50
892880,27.341695,0.003046,2023-05-31 06:05:47
892881,27.341695,0.003046,2023-05-31 08:07:59


In [164]:
electric["Электросистема. Напряжение"].value_counts()

Электросистема. Напряжение
27.341695    1042840
28.100000     248959
28.150000     175224
27.950000     162490
28.000000     162462
28.050000     133098
27.750000     128103
27.900000     118089
27.700000      99625
27.850000      99138
27.800000      81147
27.550000      69896
28.200000      67133
27.500000      65822
27.650000      65701
27.600000      58474
28.250000      54721
28.300000      38160
27.450000      33592
28.450000      29418
27.400000      28338
28.350000      23098
28.400000      20415
14.100000      19750
27.350000      11442
28.500000      11339
14.150000       8079
14.050000       7916
12.700000       7867
12.750000       7256
27.250000       5665
27.300000       5497
12.800000       4952
12.650000       4600
27.150000       3604
12.850000       3461
27.200000       3356
12.900000       2366
27.100000       2348
12.600000       2282
27.050000       1891
12.950000       1723
27.000000       1420
13.000000       1303
12.550000       1272
28.550000       1255
26.8000

In [165]:
electric.rename(columns={
    "Электросистема. Напряжение":"y"
}, inplace = True)

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_42345/4080570511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electric.rename(columns={


In [166]:
electric_train = electric.iloc[:100000, :]
split_point = int(len(transmission_train) * 0.8)

# Делим на тренировочный и тестовый наборы
train_df = electric_train.iloc[:split_point]
test_df = electric_train.iloc[split_point:]

# Выводим размеры тренировочного и тестового наборов
print(f'Train set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Train set size: 80000
Test set size: 20000


In [167]:
model = Prophet()
model.fit(train_df)
forecast = model.predict(test_df) 
print(calculate_forecast_accuracy(test_df["y"], forecast["yhat"]))

03:27:34 - cmdstanpy - INFO - Chain [1] start processing
03:27:58 - cmdstanpy - INFO - Chain [1] done processing


{'MAE': 0.4979850085162464, 'MSE': 0.29063319873539767, 'MAPE': nan}
